## Loading Data

In [1]:
import os
import ast
import typing
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from datetime import date, datetime

pd.set_option('display.max_columns', None)

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

/Users/nil.gordillo/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
months = ['Gener', 'Febrer', 'Marc', 'Abril', 'Maig', 'Juny', 'Juliol', 'Agost', 'Setembre', 'Octubre', 'Novembre', 'Desembre']

In [ ]:
os.makedirs('./bicing', exist_ok=True)

if not len(os.listdir('./bicing')):
    i2m = list(zip(range(1,13), months))
    for year in [2023, 2022, 2021, 2020, 2019]:
        for month, month_name in i2m:        
            os.system(f"wget 'https://opendata-ajuntament.barcelona.cat/resources/bcn/BicingBCN/{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
            os.system(f"7z x '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
            os.system(f"mv '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.csv' './bicing' ")
            os.system(f"rm '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")

--2023-06-29 09:03:58--  https://opendata-ajuntament.barcelona.cat/resources/bcn/BicingBCN/2023_01_Gener_BicingNou_ESTACIONS.7z
Resolving opendata-ajuntament.barcelona.cat (opendata-ajuntament.barcelona.cat)... 52.169.230.63
Connecting to opendata-ajuntament.barcelona.cat (opendata-ajuntament.barcelona.cat)|52.169.230.63|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22666599 (22M) [application/x-7z-compressed]
Saving to: ‘2023_01_Gener_BicingNou_ESTACIONS.7z’

     0K .......... .......... .......... .......... ..........  0%  840K 26s
    50K .......... .......... .......... .......... ..........  0% 1.06M 23s
   100K .......... .......... .......... .......... ..........  0% 82.1M 16s
   150K .......... .......... .......... .......... ..........  0% 1.10M 16s
   200K .......... .......... .......... .......... ..........  1% 1.11M 17s
   250K .......... .......... .......... .......... ..........  1% 4.17M 15s
   300K .......... .......... .......... ....

  5150K .......... .......... .......... .......... .......... 23% 1.19M 6s
  5200K .......... .......... .......... .......... .......... 23% 5.62M 6s
  5250K .......... .......... .......... .......... .......... 23% 80.3M 6s
  5300K .......... .......... .......... .......... .......... 24% 15.7M 6s
  5350K .......... .......... .......... .......... .......... 24% 15.6M 6s
  5400K .......... .......... .......... .......... .......... 24%  194M 6s
  5450K .......... .......... .......... .......... .......... 24% 10.5M 6s
  5500K .......... .......... .......... .......... .......... 25% 1.90M 6s
  5550K .......... .......... .......... .......... .......... 25% 2.84M 6s
  5600K .......... .......... .......... .......... .......... 25% 23.2M 6s
  5650K .......... .......... .......... .......... .......... 25% 94.1M 6s
  5700K .......... .......... .......... .......... .......... 25% 25.9M 6s
  5750K .......... .......... .......... .......... .......... 26% 45.7M 6s
  5800K ....

 12400K .......... .......... .......... .......... .......... 56%  750K 2s
 12450K .......... .......... .......... .......... .......... 56% 72.9M 2s
 12500K .......... .......... .......... .......... .......... 56%  221M 2s
 12550K .......... .......... .......... .......... .......... 56%  201M 2s
 12600K .......... .......... .......... .......... .......... 57% 4.51M 2s
 12650K .......... .......... .......... .......... .......... 57%  151M 2s
 12700K .......... .......... .......... .......... .......... 57%  156M 2s
 12750K .......... .......... .......... .......... .......... 57% 4.54M 2s
 12800K .......... .......... .......... .......... .......... 58% 95.2M 2s
 12850K .......... .......... .......... .......... .......... 58% 71.1M 2s
 12900K .......... .......... .......... .......... .......... 58% 33.4M 2s
 12950K .......... .......... .......... .......... .......... 58% 14.7M 2s
 13000K .......... .......... .......... .......... .......... 58%  101M 2s
 13050K ....

 20550K .......... .......... .......... .......... .......... 93% 3.83M 0s
 20600K .......... .......... .......... .......... .......... 93%  198M 0s
 20650K .......... .......... .......... .......... .......... 93%  202M 0s
 20700K .......... .......... .......... .......... .......... 93%  110M 0s
 20750K .......... .......... .......... .......... .......... 93% 14.0M 0s
 20800K .......... .......... .......... .......... .......... 94%  276M 0s
 20850K .......... .......... .......... .......... .......... 94%  114M 0s
 20900K .......... .......... .......... .......... .......... 94%  159M 0s
 20950K .......... .......... .......... .......... .......... 94% 15.1M 0s
 21000K .......... .......... .......... .......... .......... 95% 2.89M 0s
 21050K .......... .......... .......... .......... .......... 95% 27.7M 0s
 21100K .......... .......... .......... .......... .......... 95% 46.0M 0s
 21150K .......... .......... .......... .......... .......... 95% 14.2M 0s
 21200K ....


7-Zip [64] 17.05 : Copyright (c) 1999-2021 Igor Pavlov : 2017-08-28
p7zip Version 17.05 (locale=utf8,Utf16=on,HugeFiles=on,64 bits,8 CPUs LE)

Scanning the drive for archives:
1 file, 22666599 bytes (22 MiB)

Extracting archive: 2023_01_Gener_BicingNou_ESTACIONS.7z
--
Path = 2023_01_Gener_BicingNou_ESTACIONS.7z
Type = 7z
Physical Size = 22666599
Headers Size = 178
Method = LZMA2:24
Solid = -
Blocks = 1

Everything is Ok

Size:       265053242
Compressed: 22666599


...

HTTP request sent, awaiting response... 200 OK
Length: 24717623 (24M) [application/x-7z-compressed]
Saving to: ‘2022_01_Gener_BicingNou_ESTACIONS.7z’

     0K .......... .......... .......... .......... ..........  0%  965K 25s
    50K .......... .......... .......... .......... ..........  0% 1017K 24s
   100K .......... .......... .......... .......... ..........  0% 75.9M 16s
   150K .......... .......... .......... .......... ..........  0% 1.05M 18s
   200K .......... .......... .......... .......... ..........  1%  659K 21s
   250K .......... .......... .......... .......... ..........  1% 86.4M 18s
   300K .......... .......... .......... .......... ..........  1%  727K 20s
   350K .......... .......... .......... .......... ..........  1% 1.11M 20s
   400K .......... .......... .......... .......... ..........  1% 82.1M 18s
   450K .......... .......... .......... .......... ..........  2% 1.06M 18s
   500K .......... .......... .......... .......... ..........  2% 90.8M 16s
  

  5400K .......... .......... .......... .......... .......... 22% 1.27M 6s
  5450K .......... .......... .......... .......... .......... 22%  203M 6s
  5500K .......... .......... .......... .......... .......... 22% 36.1M 6s
  5550K .......... .......... .......... .......... .......... 23% 16.2M 6s
  5600K .......... .......... .......... .......... .......... 23% 10.9M 6s
  5650K .......... .......... .......... .......... .......... 23% 12.9M 6s
  5700K .......... .......... .......... .......... .......... 23% 1.13M 6s
  5750K .......... .......... .......... .......... .......... 24% 90.9M 6s
  5800K .......... .......... .......... .......... .......... 24% 54.4M 6s
  5850K .......... .......... .......... .......... .......... 24% 18.5M 6s
  5900K .......... .......... .......... .......... .......... 24% 81.2M 6s
  5950K .......... .......... .......... .......... .......... 24% 27.7M 6s
  6000K .......... .......... .......... .......... .......... 25% 16.7M 6s
  6050K ....

 13050K .......... .......... .......... .......... .......... 54% 1.61M 2s
 13100K .......... .......... .......... .......... .......... 54%  183M 2s
 13150K .......... .......... .......... .......... .......... 54% 29.4M 2s
 13200K .......... .......... .......... .......... .......... 54% 89.8M 2s
 13250K .......... .......... .......... .......... .......... 55% 16.0M 2s
 13300K .......... .......... .......... .......... .......... 55% 5.26M 2s
 13350K .......... .......... .......... .......... .......... 55% 57.7M 2s
 13400K .......... .......... .......... .......... .......... 55% 41.9M 2s
 13450K .......... .......... .......... .......... .......... 55% 71.7M 2s
 13500K .......... .......... .......... .......... .......... 56% 23.5M 2s
 13550K .......... .......... .......... .......... .......... 56%  115M 2s
 13600K .......... .......... .......... .......... .......... 56% 29.9M 2s
 13650K .......... .......... .......... .......... .......... 56% 87.7M 2s
 13700K ....

 18850K .......... .......... .......... .......... .......... 78% 8.05M 1s
 18900K .......... .......... .......... .......... .......... 78% 18.1M 1s
 18950K .......... .......... .......... .......... .......... 78%  267M 1s
 19000K .......... .......... .......... .......... .......... 78% 47.4M 1s
 19050K .......... .......... .......... .......... .......... 79% 9.21M 1s
 19100K .......... .......... .......... .......... .......... 79% 21.3M 1s
 19150K .......... .......... .......... .......... .......... 79% 7.91M 1s
 19200K .......... .......... .......... .......... .......... 79%  210M 1s
 19250K .......... .......... .......... .......... .......... 79% 78.4M 1s
 19300K .......... .......... .......... .......... .......... 80% 11.4M 1s
 19350K .......... .......... .......... .......... .......... 80% 2.90M 1s
 19400K .......... .......... .......... .......... .......... 80% 96.7M 1s
 19450K .......... .......... .......... .......... .......... 80% 59.8M 1s
 19500K ....


7-Zip [64] 17.05 : Copyright (c) 1999-2021 Igor Pavlov : 2017-08-28
p7zip Version 17.05 (locale=utf8,Utf16=on,HugeFiles=on,64 bits,8 CPUs LE)

Scanning the drive for archives:
1 file, 24717623 bytes (24 MiB)

Extracting archive: 2022_01_Gener_BicingNou_ESTACIONS.7z


In [ ]:
df = pd.DataFrame()

In [ ]:
def get_datetime(miliseconds: int):
    return datetime.fromtimestamp(miliseconds)

def create_date_df(df: pd.DataFrame):
    df['date'] = pd.to_datetime(
        df['last_reported'].apply(lambda x: get_datetime(x))
    )
    
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['hour'] = df['date'].dt.hour
    df['day'] = df['date'].dt.day
    
    return df

In [ ]:
def create_date_time(row):
    return datetime(
        int(row['year']),
        int(row['month']),
        int(row['day']),
        int(row['hour'])
    )

In [ ]:
def process_new_df(new_df: pd.DataFrame):
    # Unique values
    new_df = new_df.drop_duplicates()
    new_df = new_df.dropna(subset=['last_reported', 'last_updated'], axis=0)
    new_df = new_df.sort_values('last_reported', ascending=True)
    
    # Convert some categorical into numerical
    new_df.status = np.where(new_df.status == 'IN_SERVICE', 1, 0)
    new_df.is_charging_station = np.where(new_df.is_charging_station, 1, 0)
    
    # Create the dates from timestamp and group statistics
    new_df = create_date_df(new_df)
    new_df = new_df\
        .groupby(['station_id', 'year', 'month', 'day', 'hour'])\
        .mean(numeric_only=True)\
        .reset_index()
    
    # Creation of the dates
    new_df['date_time'] = new_df.apply(lambda row: create_date_time(row), axis=1)
    new_df['date'] = new_df.date_time.dt.date
    
    # Create percentage of docks availability
    new_df['percentage_docks_available'] = new_df['num_bikes_available'] / new_df['num_docks_available']

    return new_df

In [ ]:
def get_df_processed(file_name_csv: str = 'df_processed.csv'):
    if os.path.exists(file_name_csv):
        return pd.read_csv(file_name_csv)
        
    for csv in tqdm(os.listdir('./bicing')):
        new_df = pd.read_csv('./bicing/' + csv)
        new_df = process_new_df(new_df)

        # Filter out incorrect dates
        year, month, *_ = csv.split('_')
        year, month = int(year), int(month)
        new_df = new_df[(new_df.year == year) & (new_df.month == month)]

        df = pd.concat(
            [df, new_df], 
            axis=0
        )
    
    df.to_csv(file_name_csv, index=False, header=True)
    return df

df = get_df_processed()

In [ ]:
# Handle NaN: Assume that the NaN in traffic is 0
df.traffic = df.traffic.replace(np.nan, 0)

In [ ]:
df.shape

In [ ]:
df.head()

### Add station information

In [ ]:
def get_station_json(
    url: str = 'https://opendata-ajuntament.barcelona.cat/data/dataset/bd2462df-6e1e-4e37-8205-a4b8e7313b84/resource/e5adca8d-98bf-42c3-9b9c-364ef0a80494/download'
):
    
    res = requests.get(url)
    return res.json()

In [ ]:
station_json = get_station_json()

In [ ]:
station_df = pd.DataFrame(station_json['data']['stations'])
station_df.head()

In [ ]:
# Some fields are not going to be used
station_drop_fields = [
    'physical_configuration',          # unique value: ELECTRICBIKESTATION
    '_ride_code_support',              # unique value: all True
    'nearby_distance',                 # unique value: 1000
    'name', 'address', 'post_code',    # too specific to each of the stations
    'is_charging_station',             # already in the main df
    'rental_uris', 'cross_street',     # vast majority are none
    
]

In [ ]:
station_df = station_df.drop(station_drop_fields, axis=1)
station_df.head()

In [ ]:
df = pd.merge(df, station_df)

In [ ]:
df.head()

### Add Weather information

Add information relative to the wather ([source](https://www.visualcrossing.com/weather/weather-data-services#))


In [ ]:
print('Start date: ', df['date'].min())
print('End date: ', df['date'].max())

In [ ]:
def weather_api_extraction(url: str) -> pd.DataFrame:
    response = requests.get(url)
    response_json = response.json()
    
    weather_df = pd.json_normalize(response_json['days'])
    
    return weather_df

In [ ]:
def weather_df_drop_columns(
    weather_df: pd.DataFrame, 
    columns_to_drop: typing.List[str] = [
        'datetimeEpoch', 
        'tzoffset', 
        'source',
        'name',                     # all barcelona
        'stations',                 # which stations tracked that weather
        'severerisk'                # all nulls
        'precipprob',               # 100 if it rains, 0 otherwise. Doesn't add new info
        'preciptype',               # 'rain' if it rains, 'snow' if it snows. Doesn't add new info
        'sunrise', 'sunset',        # simplify variables, too specific
        'description',              # simplify variables, too specific
        'sunrise',                  # simplify variables, too specific
        'sunset',                   # simplify variables, too specific
        'moonphase',                # simplify variables, too specific
        'sealevelpressure',         # simplify variables, too specific
    ]
):
    
    return weather_df.drop(columns_to_drop, axis=1, errors='ignore')

In [ ]:
def process_weather_df(weather_df: pd.DataFrame) -> pd.DataFrame:
    weather_df.datetime = pd.to_datetime(weather_df.datetime)
    weather_df['date'] = weather_df.datetime.dt.date
    
    return weather_df

In [ ]:
os.makedirs('./weather', exist_ok=True)

if not len(os.listdir('./weather')):
    weather_url1 = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/barcelona/2019-03-28/2021-11-30?unitGroup=metric&include=days&key=4T9KXABWNUV92K2WTZMA7JXZ3&contentType=json'
    weather_df1 = weather_api_extraction(weather_url1)
    weather_df1.to_csv('./weather/weather_2019_03_28_to_2021_11_30.csv', index=False, header=True)
    
    weather_url2 = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/barcelona/2021-12-01/2023-04-30?unitGroup=metric&include=days&key=6NT6N8CRPMNTAEE2FH2EQZ98U&contentType=json'
    weather_df2 = weather_api_extraction(weather_url2)
    weather_df2.to_csv('./weather/weather_2021_12_01_to_2023_04_30.csv', index=False, header=True)

else:
    weather_df1 = pd.read_csv('./weather/weather_2019_03_28_to_2021_11_30.csv')
    weather_df2 = pd.read_csv('./weather/weather_2021_12_01_to_2023_04_30.csv')   

In [ ]:
weather_df = pd\
    .concat([weather_df1, weather_df2], axis=0)\
    .reset_index(drop=True)

In [ ]:
weather_df = weather_df_drop_columns(weather_df)
weather_df = process_weather_df(weather_df)

In [ ]:
weather_df.head()

In [ ]:
df.date = pd.to_datetime(df.date).dt.date
weather_df.date = pd.to_datetime(weather_df.date).dt.date

df = pd.merge(df, weather_df, left_on='date', right_on='date')

In [ ]:
df.head()

### Add Covid Information

The daily covid cases that there were in Barcelona ([source](https://www.amb.cat/en/web/area-metropolitana/dades-obertes/cataleg/detall/-/dataset/covid-19-positive-cases/9147624/11692))

In [ ]:
def get_covid_json(
    url: str = "https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search_sql?sql=SELECT%20*%20from%20%22f627ac0a-d05f-416d-9773-eeb464a3fc44%22%20WHERE%20%22Nom_Indicador%22%20LIKE%20%27Casos%20de%20COVID-19%20a%20Barcelona%20(diari)%27"
):
    
    res_covid = requests.get(url)
    return res_covid.json()['result']['records']

In [ ]:
covid_json = get_covid_json()

In [ ]:
covid_df = pd.DataFrame(covid_json)
covid_df.head()

In [ ]:
covid_df.shape

In [ ]:
covid_df = covid_df[['Data_Indicador', 'Valor']]
covid_df.head()

In [ ]:
covid_df['date'] = pd.to_datetime(covid_df.Data_Indicador).dt.date

In [ ]:
covid_df = covid_df\
    .drop('Data_Indicador', axis=1)\
    .rename(columns={'Valor': 'covid_cases'})

In [ ]:
covid_df.head()

In [ ]:
df = pd.merge(
    df, 
    covid_df, 
    how='left', 
    left_on='date', 
    right_on='date'
)

We only have NaN values afterwards (not previous to the start of Covid).

For those NaN, we will replace them by a percentile that is at the bottom (still not 0, as there are cases).

In [ ]:
df.covid_cases = df.covid_cases.astype('float')

In [ ]:
df.covid_cases.describe()

In [ ]:
df.covid_cases.hist(bins=50)

In [ ]:
def fill_with_percentile(
    df: pd.DataFrame, 
    column: str, 
    percentile: float = 0.03,
    logging: bool = True
):
    
    quantile_value = df[column].quantile(percentile)
    
    if logging:
        print(f'Percentile {percentile} value is: ', quantile_value)
        
    df[column] = df[column].fillna(quantile_value)
    
    return df

In [ ]:
df = fill_with_percentile(df, 'covid_cases')

We could add also information about in which dates the people where closed into their homes.

In [ ]:
confinment_start = '2020-03-14'
confinment_date_start = datetime.strptime(confinment_start, '%Y-%m-%d').date()

In [ ]:
confinment_end = '2020-06-21'
confinment_date_end = datetime.strptime(confinment_start, '%Y-%m-%d').date()

In [ ]:
index_non_confinment = df[
    (df['date'] < confinment_date_start) |
    (df['date'] > confinment_date_end)
].index

In [ ]:
    df.loc[index_non_confinment, 'confinment'] = 0
df['confinment'] = df['confinment'].fillna(1)
df['confinment'] = df['confinment'].astype('int')

### Barcelona Events

In [ ]:
os.makedirs('./events', exist_ok=True)

In [ ]:
def get_opendataevents_json(url: str = 'https://www.bcn.cat/tercerlloc/files/opendatabcn_agenda.json'):
    res = requests.get(url)
    return res.json()

In [ ]:
if not os.path.isfile('./events/opendatabcn_agenda.json'):
    events_json = get_opendataevents_json()
    events_df = pd.DataFrame(events_json)
    events_df.to_csv('./events/opendatabcn_agenda.json', index=False, header=True)

else:
    events_df = pd.read_csv('./events/opendatabcn_agenda.json')

In [ ]:
events_df.head()

In [ ]:
# Only having active dates
events_df = events_df[events_df.core_type_name != 'Equipament']
events_df = events_df[events_df.event_status == 'ok']

In [ ]:
def events_df_drop_columns(
    events_df: pd.DataFrame, 
    columns_to_drop: typing.List[str] = [
        'register_id', 'prefix', 'suffix', 'name', 'created', 'modified', 'status', 'status_name',     # too specific
        'core_type', 'core_type_name',                                                                 # most is only one type of value
        'body', 'tickets_data', 'addresses', 'entity_types_data', 'attribute_categories', 'values',
        'from_relationships', 'to_relationships', 'classifications_data', 'secondary_filters_data',
        'timetable', 'image_data', 'gallery_data', 'warnings',                                         # unuseful information
        'is_section_of_data', 'sections_data', 'estimated_dates', 'languages_data', 'type', 'type_name',
        'period', 'period_name', 'event_status_name', 'event_status', 'ical',
        'geo_epgs_25831', 'geo_epgs_23031'                                                             # unwanted loations
    ]
):
    
    return events_df.drop(columns_to_drop, axis=1, errors='ignore')

In [ ]:
# Drop columns not wanted
events_df = events_df_drop_columns(events_df)

In [ ]:
# Drop events with not enough data
events_df.dropna(
    axis=0, 
    how='any', 
    subset=['geo_epgs_4326', 'start_date'], 
    inplace=True
)

In [ ]:
events_df.head()

In [ ]:
def get_event_geographics(geo_position: str):
    geo_position = ast.literal_eval(geo_position)
    return geo_position['x'], geo_position['y']

In [ ]:
events_geographic_x_y = events_df.geo_epgs_4326.apply(lambda x: get_event_geographics(x))
events_df['x'] = [x for x, _ in events_geographic_x_y]
events_df['y'] = [y for _, y in events_geographic_x_y]

events_df = events_df[events_df.x > 0]
events_df = events_df[events_df.y > 0]

events_df.drop(['geo_epgs_4326'], axis=1, inplace=True)

In [ ]:
# In the case of null values of end date, we will assume is the same as start date (we will consider only days)
events_df.end_date.fillna(events_df.start_date, inplace=True)

In [ ]:
events_df['start_date'] = pd.to_datetime(events_df['start_date'], utc=True).dt.date
events_df['end_date'] = pd.to_datetime(events_df['end_date'], utc=True).dt.date

In [ ]:
# Filter out by maximum and minimum dates of df
df_min_date, df_max_date = df.date.min(), df.date.max()

events_df = events_df[events_df.start_date > df_min_date]
events_df = events_df[events_df.end_date < df_max_date]

In [ ]:
print(events_df.shape)
events_df.head()

In [ ]:
sns.jointplot(x="x", y="y", data=events_df, alpha=0.2)

In [ ]:
std_dev_x, std_dev_y = events_df.x.std(), events_df.y.std()

In [ ]:
# Filter df by the dates that we have
df_common_events = pd.concat(
    [
        pd.merge(df, events_df, left_on='date', right_on='start_date'), 
        pd.merge(df, events_df, left_on='date', right_on='end_date')
    ],
    axis=0
).drop_duplicates()

In [ ]:
print(df_common_events.shape)
df_common_events.head()

In [ ]:
with tqdm(total=len(events_df)) as pbar:
    for idx, row in events_df.iterrows():    
        # Filter by the range of the dates
        x, y = row.x, row.y
        start_date, end_date = row['start_date'], row['end_date']
        range_dates = pd.date_range(start_date, end_date)
        range_df = df_common_events[df_common_events.date.isin(range_dates)]
        
        if len(range_df) > 0:
            # Near position of the event with relation of the biking center (30% of standard deviations of x & y)
            range_df = range_df[(range_df.lat > x - std_dev_x * 0.3) & (range_df.lat < x + std_dev_x * 0.3)]
            range_df = range_df[(range_df.lat > y - std_dev_y * 0.3) & (range_df.lat < y + std_dev_y * 0.3)]
            df.loc[range_df.index, 'has_event'] = 1

        pbar.update(1)
    

In [ ]:
df.has_event = df.has_event.fillna(0)

In [ ]:
df.head()

In [ ]:
df.columns.values

Once we have extended our dataframe with external sources, we are going to make the selection of the columns that we want.

In [ ]:
selected_columns_df = [
    'station_id', 
    'num_bikes_available', 'num_bikes_available_types.mechanical',                            # docks info
    'num_bikes_available_types.ebike', 'num_docks_available', 'percentage_docks_available',   # docks info
    'status', 'lat', 'lon', 'altitude', 'capacity',                                           # station info
    'year', 'month', 'day', 'hour', 'date_time', 'date',                                      # time info
    'traffic', 'covid_cases', 'confinment', 'has_event',                                      # context info
    'temp', 'feelslike', 'humidity', 'precip', 'snow', 'windspeed', 'uvindex', 'visibility',  # meteorological info
]

selected_df = df[selected_columns_df]

In [ ]:
# Checkpoint of the extended df with external sources
selected_df.to_csv(
    'df_selected.csv.gz', 
    index=False, 
    header=True, 
    compression='gzip'
)

## Data Engineering

Engineer new variables from the information that we already have. 

In [ ]:
selected_df.date_time = pd.to_datetime(selected_df.date_time)

### Time

In [ ]:
# Would need to get the dummies for them
date_time_fields = ['day_info', 'hour_info', 'month_info']

Related to the **day** (weekend).

In [ ]:
def get_day_info(date_time):
    return 'weekend' if date_time.weekday() in [5, 6] else 'weekday'

In [ ]:
selected_df['day_info'] = selected_df.date_time.apply(lambda x: get_day_info(x))

Related to the **hour** (which time of day it was).

In [ ]:
def get_hour_info(date_time):
    if date_time.hour in range(5):
        return 'late_night'

    elif date_time.hour in range(5, 9):
        return 'early_morning'

    elif date_time.hour in range(9, 13):
        return 'morning'
    
    elif date_time.hour in range(13, 17):
        return 'noon'
    
    elif date_time.hour in range(17, 21):
        return 'eve'
    
    return 'night'

In [ ]:
selected_df['hour_info'] = selected_df.date_time.apply(lambda x: get_hour_info(x))

Related to the **month** (season).

In [ ]:
def get_month_info(date):
    if date.month in (3, 4, 5):
        return 'spring'
    
    elif date.month in (6, 7, 8):
        return 'summer'
    
    elif date.month in (9, 10, 11):
        return 'autumn'
    
    return 'winter'

In [ ]:
selected_df['month_info'] = selected_df.date_time.apply(lambda x: get_month_info(x))

In [ ]:
selected_df.head()

### Data Shifting

We need to create a shift of the availability in the previous availabilities.

In [ ]:
selected_df = selected_df.sort_values(['station_id', 'date_time'], ascending=True)

In [ ]:
df_shifted = pd.DataFrame()

In [ ]:
for station_id in tqdm(selected_df.station_id.unique()):
    station_df = selected_df[selected_df.station_id == station_id]
    station_df['ctx-4'] = station_df.percentage_docks_available.shift(4)
    station_df['ctx-3'] = station_df.percentage_docks_available.shift(3)
    station_df['ctx-2'] = station_df.percentage_docks_available.shift(2)
    station_df['ctx-1'] = station_df.percentage_docks_available.shift(1)
    
    df_shifted = pd.concat([station_df, df_shifted], axis=0)

In [ ]:
df_shifted.head()

### Train / Val / Test Split

In [ ]:
df_shifted.head()

In [ ]:
train_df = df_shifted[df_shifted.year != 2023]
test_df = df_shifted[df_shifted.year == 2023]

In [ ]:
print('Train size: ', train_df.shape[0])
print('Test size: ', test_df.shape[0])

In [ ]:
valid_df = test_df[test_df.month <= 3]
test_df = test_df[test_df.month > 3]

### Exploration